In [1]:
%pip install azure-cognitiveservices-vision-customvision
%pip install pylabel

#Import Azure cognitive services libraries 
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

#Import other libraries used in this notebook 
import os, zipfile
from pathlib import PurePath
from os.path import exists
from decimal import *

from pylabel import importer


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Replace with your Azure endpoint and subscription keys.
ENDPOINT = "https://eilearn.cognitiveservices.azure.com/"
training_key = "b4df3e4c129d481e9257ef97ab1bf51f"
prediction_key = "3d36c550ca484853b4f94a01293abf01"
prediction_resource_id = "e3ad05bf-f0e5-4124-967f-5a245408d944"

In [3]:
#Initialize objects used by Azure Congitive vision
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [4]:
#Create a new project
publish_iteration_name = "ign_france"
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")
project = trainer.create_project(
    "ign_france", domain_id=obj_detection_domain.id)
#If you browse to https://www.customvision.ai/ you should see a new project called "PyLabel Sample Dataset"

In [4]:
# Get an existing project
project = trainer.get_project(
    project_id="90de1015-742b-462c-9bb3-69646557d679")

# print project name 
print(project.name)

ign_france


## Upload images

In [11]:
#Import annotations as a PyLabel dataset
dataset = importer.ImportVOC(path=r"C:\Users\jaddh\code_projects\EILearn\map_layers\ign_france\images\labeled",
                             path_to_images=r"C:\Users\jaddh\code_projects\EILearn\map_layers\ign_france\images\labeled")
dataset.df.head(10000)

Importing VOC files...: 100%|██████████| 474/474 [00:00<00:00, 4218.53it/s]


,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_iscrowd,ann_keypoints,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,C:\Users\jaddh\code_projects\EILearn\map_layer...,0.jpg,NaN,1,1024,1024,3,0,891.0,167.0,...,NaN,NaN,Unspecified,0,0,0,eiland,NaN,,1
1,C:\Users\jaddh\code_projects\EILearn\map_layer...,1.jpg,NaN,3,1024,1024,3,0,412.0,394.0,...,NaN,NaN,Unspecified,0,0,0,eiland,NaN,,1
2,C:\Users\jaddh\code_projects\EILearn\map_layer...,105.jpg,NaN,5,1024,1024,3,0,244.0,47.0,...,NaN,NaN,Unspecified,0,0,0,eiland,NaN,,1
3,C:\Users\jaddh\code_projects\EILearn\map_layer...,106.jpg,NaN,7,1024,1024,3,0,575.0,195.0,...,NaN,NaN,Unspecified,0,0,0,eiland,NaN,,1
4,C:\Users\jaddh\code_projects\EILearn\map_layer...,106.jpg,NaN,7,1024,1024,3,0,208.0,570.0,...,NaN,NaN,Unspecified,0,0,0,eiland,NaN,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,C:\Users\jaddh\code_projects\EILearn\map_layer...,93.jpg,NaN,469,1024,1024,3,0,796.0,805.0,...,NaN,NaN,Unspecified,0,0,0,eiland,NaN,,1
332,C:\Users\jaddh\code_projects\EILearn\map_layer...,98.jpg,NaN,471,1024,1024,3,0,847.0,408.0,...,NaN,NaN,Unspecified,0,0,0,eiland,NaN,,1
333,C:\Users\jaddh\code_projects\EILearn\map_layer...,98.jpg,NaN,471,1024,1024,3,0,622.0,654.0,...,NaN,NaN,Unspecified,0,0,0,eiland,NaN,,1


## Import to Azure Custom Vision
PyLabel stores the annotations as a pandas dataframe. Now you can use extract the annotations from the dataframe and use it as inputs to the Custom Vision APIs. 

The first step is to create tags for each of the classes in your custom dataset. A list of class names is available in the dataset.analyze.classes property. 

In [6]:
print(dataset.analyze.classes)
#Create a tag for each class and store then in a dict where the class name is the key
tags = {}
for class_name in dataset.analyze.classes:
    tag = trainer.create_tag(project.id, class_name)
    tags[class_name] = tag

['eiland']


In [7]:
tags = {}

for class_name in dataset.analyze.classes:
    tags[class_name] = trainer.get_tags(project.id)[0]


Now if you check your account on https://www.customvision.ai/ you should see a new project called "PyLabel Sample Dataset" with 2 tags added: Squirrels and Nuts. 

You are ready to upload your images and annotations. For each image in your dataset you will need to add "Regions" for each bounding box and then upload the image and annotations. 

In [12]:
#Iterate the rows for each image in the dataframe
for img_filename, img_df in dataset.df.groupby('img_filename'):
    img_path = PurePath(r"C:\Users\jaddh\code_projects\EILearn\map_layers\ign_france\images\labeled", img_filename)
    assert exists(img_path), f"File does not exist: {img_path}"

    #Create a region object for each bounding box in the dataset 
    regions = []
    for index, row in img_df.iterrows():
        print(row.img_filename)

        #Normalize the boundings box coordinates between 0 and 1
        x = Decimal(row.ann_bbox_xmin / row.img_width).min(1)
        y = Decimal(row.ann_bbox_ymin / row.img_height).min(1)
        w = Decimal(row.ann_bbox_width / row.img_width).min(1-x)
        h = Decimal(row.ann_bbox_height / row.img_height).min(1-y)
        
        regions.append(Region(
                tag_id=tags[row.cat_name].id, 
                left=x,
                top=y,
                width=w,
                height=h
            )
        )

    #Create an object with the image and all of the annotations for that image
    with open(img_path, mode="rb") as image_contents:
        image_and_annotations = [ImageFileCreateEntry(name=img_filename, contents=image_contents.read(), regions=regions)]

    #Upload the image and all annnotations for that image
    upload_result = trainer.create_images_from_files(
            project.id, 
            ImageFileCreateBatch(images=image_and_annotations)
        )
    
    #If upload is not successful, print details about that image for debugging 
    if not upload_result.is_batch_successful:
        print("Image upload failed.")
        for image in upload_result.images:
            print(img_path)
            print("Image status: ", image.status)
            print(regions)

#This will take a few minutes 
print("Upload complete")

0.jpg
Image upload failed.
C:\Users\jaddh\code_projects\EILearn\map_layers\ign_france\images\labeled\0.jpg
Image status:  OKDuplicate
1.jpg
Image upload failed.
C:\Users\jaddh\code_projects\EILearn\map_layers\ign_france\images\labeled\1.jpg
Image status:  OKDuplicate
105.jpg
Image upload failed.
C:\Users\jaddh\code_projects\EILearn\map_layers\ign_france\images\labeled\105.jpg
Image status:  OKDuplicate
106.jpg
106.jpg
Image upload failed.
C:\Users\jaddh\code_projects\EILearn\map_layers\ign_france\images\labeled\106.jpg
Image status:  OKDuplicate
[<azure.cognitiveservices.vision.customvision.training.models._models_py3.Region object at 0x0000023FB221DB50>, <azure.cognitiveservices.vision.customvision.training.models._models_py3.Region object at 0x0000023FB2204250>]
107.jpg
107.jpg
107.jpg
Image upload failed.
C:\Users\jaddh\code_projects\EILearn\map_layers\ign_france\images\labeled\107.jpg
Image status:  OKDuplicate
[<azure.cognitiveservices.vision.customvision.training.models._models_p